In [94]:
import shutil
import pathlib
import glob
import re
import zipfile
import os
from google.cloud import storage
import yaml
import json
import subprocess

In [117]:
with open('../config/gcp.yaml','r') as f:
    gcp = yaml.load(f)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=gcp['REDENTIALS_JSON']
project_id = gcp['GCP_PROJECT']

In [118]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='C:\\Users\\hajime kobayashi\\My First Project-ea55bbd81537.json'
p_temp = pathlib.Path('../functions')
function_list=[p for p in p_temp.iterdir() if p.is_dir()]
with open('../config/function_deploy.yaml','r') as f:
    conf = yaml.load(f)

In [119]:
conf

{'FUNCTION_BUCKET': 'sys_functions11'}

In [133]:
#FunctionsをDeploy
with open("../config/functions_args.json") as f:
    function_args_set = json.load(f)
deploy_func_set = []
gcloud_cmmand = ["gcloud","functions","deploy"]
for deploy_func in function_args_set:
    deploy_func_set.append(list(deploy_func.keys())[0])

In [134]:
type(function_args_set)

list

In [135]:
deploy_func_set

['week_race_get',
 'predict_race_get',
 'predict_horse_get',
 'past_race_get',
 'predict_race_check',
 'tweet',
 'predict_result_aggregate',
 'predict_result_output']

In [136]:
#バケット名を指定してbucketを取得
client = storage.Client(project_id)
bucket_name = conf['FUNCTION_BUCKET']

bucket = client.get_bucket(bucket_name)

for function in function_list:
    if function.name in deploy_func:
        print("Not Deploy {} ".format(function.name))

    gcloud_cmmand = ["gcloud","functions","deploy"]   
    zip_file_name = "{}.zip".format(function.name)
    print(zip_file_name)
    zip_file_list = [p for p in function.glob('**/*') if 'yaml' not in str(p)]
    with zipfile.ZipFile(zip_file_name, 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
        for i in zip_file_list:
            new_zip.write(str(i),arcname=i.name)
    #storageにアップロード
    #blob = bucket.blob(zip_file_name)
    #blob.upload_from_filename(zip_file_name)
    #アップロード済ファイル削除
    os.remove(zip_file_name)
    gsutil_link = "gs://{}/{}".format(bucket_name,zip_file_name)
    for arg_f in function_args_set:
        #argsファイル内に存在していた場合、コマンドを設定
        
        if function.name in arg_f:
            arg_data = arg_f[function.name]
            runtime = arg_data['runtime']
            trigger= arg_data['trigger']
            if type(trigger) is dict:
                trigger_type = list(trigger.keys())[0]
                trigger_event = list(trigger.values())[0]
                trigger_command = "--trigger-{}={}".format(trigger_type,trigger_event)
            else:
                trigger_type = trigger
                trigger_command = "--trigger-{}".format(trigger_type)
            memory= arg_data['memory']
            timeout= arg_data['timeout']
            entry_point= arg_data['entry_point']
    env_file = list(function.glob('**/*.yaml'))
    if len(env_file) != 0:
        env_file = env_file[0]
    #コマンドの設定
    gcloud_cmmand.append(function.name)
    gcloud_cmmand.append("--runtime={}".format(runtime))
    gcloud_cmmand.append(trigger_command)
    gcloud_cmmand.append("--timeout={}".format(timeout))
    gcloud_cmmand.append("--memory={}".format(memory))
    gcloud_cmmand.append("--source={}".format(gsutil_link))
    gcloud_cmmand.append("--entry-point={}".format(entry_point))
    gcloud_cmmand.append("--env-vars-file={}".format(env_file))
    print(gcloud_cmmand)
    #proc = subprocess.run(gcloud_cmmand,shell =True,stdout = subprocess.PIPE, stderr = subprocess.PIPE)

    
        
    

    

past_race_get.zip
['gcloud', 'functions', 'deploy', 'past_race_get', '--runtime=python37', '--trigger-topic=past_race', '--timeout=540', '--memory=512MB', '--source=gs://sys_functions11/past_race_get.zip', '--entry-point=past_race_get', '--env-vars-file=..\\functions\\past_race_get\\env.yaml']
predict_horse_get.zip
['gcloud', 'functions', 'deploy', 'predict_horse_get', '--runtime=python37', '--trigger-topic=predict_horse', '--timeout=540', '--memory=512MB', '--source=gs://sys_functions11/predict_horse_get.zip', '--entry-point=predict_horse_get', '--env-vars-file=..\\functions\\predict_horse_get\\env.yaml']
predict_race_check.zip
['gcloud', 'functions', 'deploy', 'predict_race_check', '--runtime=python37', '--trigger-http', '--timeout=540', '--memory=512MB', '--source=gs://sys_functions11/predict_race_check.zip', '--entry-point=predict_race_check', '--env-vars-file=..\\functions\\predict_race_check\\env.yaml']
predict_race_get.zip
['gcloud', 'functions', 'deploy', 'predict_race_get', '-

In [115]:
gsutil_file = 'gs://{}/{}'.format(blob.bucket,blob.name)

In [58]:
blob.generation

1588051750197487

In [53]:
gsutil_file

'gs://<Bucket: sys_functions11>/week_race_get.zip'

In [102]:
import subprocess

In [110]:
top=['gcloud', 'functions', 'deploy', 'week_race_get', '--runtime=nodejs8', '--trigger-http', '--timeout=540', '--memory=512MB', '--source=gs://sys_functions11/week_race_get.zip', '--entry-point=week_race_get', '--env-vars-file=..\\functions\\week_race_get\\env.yaml']

print(proc)

CompletedProcess(args=['gcloud', 'functions', 'deploy', 'week_race_get', '--runtime=nodejs8', '--trigger-http', '--timeout=540', '--memory=512MB', '--source=gs://sys_functions11/week_race_get.zip', '--entry-point=week_race_get', '--env-vars-file=..\\functions\\week_race_get\\env.yaml'], returncode=0, stdout=b"availableMemoryMb: 512\r\nentryPoint: week_race_get\r\nenvironmentVariables:\r\n  TOPIC_NAME: predict_race\r\nhttpsTrigger:\r\n  url: https://us-central1-nth-computing-270722.cloudfunctions.net/week_race_get\r\ningressSettings: ALLOW_ALL\r\nlabels:\r\n  deployment-tool: cli-gcloud\r\nname: projects/nth-computing-270722/locations/us-central1/functions/week_race_get\r\nruntime: nodejs8\r\nserviceAccountEmail: nth-computing-270722@appspot.gserviceaccount.com\r\nsourceArchiveUrl: gs://sys_functions11/week_race_get.zip\r\nstatus: ACTIVE\r\ntimeout: 540s\r\nupdateTime: '2020-04-28T07:50:27.236Z'\r\nversionId: '1'\r\n", stderr=b'Allow unauthenticated invocations of new function [week_rac

In [64]:
from google.cloud import pubsub
from google.cloud import bigquery

In [66]:
c = pubsub.PublisherClient(project_name)

TypeError: __new__() takes from 1 to 4 positional arguments but 21 were given

In [51]:
topic_path = c.topic_path(project_name,"unti")
res = c.create_topic(topic_path)

In [54]:
bu = client.create_bucket('o1111ds11p',location="us-east1")

In [65]:
client = bigquery.Client(project_name)

In [83]:
import json
bigquery_table_list = conf['BIGQUERY_TABLE_LIST']
for table_data in bigquery_table_list:
    schema_file_path = table_data["schema_json_path"]
    with open(schema_file_path,'r') as f:
        schema_data = json.load(f)
    schema = [bigquery.SchemaField(sch["name"], sch["type"],mode=sch["mode"]) for sch in schema_data]


In [82]:
schema_data

[{'mode': 'NULLABLE', 'name': 'race_id', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'race_num', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'race_name', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'race_date', 'type': 'DATE'},
 {'mode': 'NULLABLE', 'name': 'race_rank1', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'race_rank2', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'distance', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'kind', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'condition', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'place', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'horse_count', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'lap_time', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'conner1', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'conner2', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'conner3', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'conner4', 'type': 'STRING'}]

In [84]:
schema

[SchemaField('result7', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('result6', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('result5', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('result1', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('result3', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('horse_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('race_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('pred_model_1', 'FLOAT', 'REPEATED', None, ()),
 SchemaField('pred_model_4', 'FLOAT', 'REPEATED', None, ()),
 SchemaField('result2', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('pred_model_3', 'FLOAT', 'REPEATED', None, ()),
 SchemaField('result4', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('pred_model_2', 'FLOAT', 'REPEATED', None, ())]